# PEFT (Parameter-Efficient Fine-Tuning)
## 前言：為什麼需要 PEFT？

學會了 SFT、DPO、PPO 等對齊方法的「目標函數」和「訓練邏輯」。但當你真正要在自己的機器上跑這些訓練時，會遇到一個殘酷的現實：

### 全參數 Fine-tuning 的資源需求

**以 LLaMA-2 7B 為例：**

- 模型參數量：7B (70億)
- 每個參數：2 bytes (fp16) 或 4 bytes (fp32)

**只是載入模型：**

- 模型權重: 7B × 2 bytes = 14 GB

**訓練時需要的記憶體：**

- 梯度: 7B × 2 bytes = 14 GB
- 優化器狀態 (AdamW): 7B × 8 bytes = 56 GB (m, v 各 4 bytes)
- 激活值 (activations): ~20-40 GB (視 batch size)
- 總計: 14 + 14 + 56 + 30 = 114 GB

**一張 A100 (80GB) 都不夠用**

**更大的模型更誇張：**

- LLaMA-2 13B: ~200 GB
- LLaMA-2 70B: ~1 TB
- GPT-3 175B: ~2.5 TB

**全參數訓練 70B 模型需要：**

- 8-16 張 A100 (80GB)
- 分散式訓練框架
- 幾萬到幾十萬美金的算力成本

### 問題不只是記憶體

1. 訓練時間長
    - 更新所有參數 → 每步更慢
    - 70B 模型一步可能要幾秒
2. 過擬合風險高
    - 可調參數太多 → 容易記住訓練資料
    - 特別是資料量小的時候
3. 部署困難
    - 每個任務要存一份完整模型
    - 10 個任務 = 10 × 14GB = 140GB
4. 訓練不穩定
    - 參數空間巨大 → 容易發散
    - 需要極小的學習率和精細調參

## PEFT 的核心理念

**核心理念：** 凍結絕大部分參數，只訓練極少量的新增參數

- Full Fine-tuning：更新 7B 參數
- PEFT：更新~10M 參數 (0.14%)
- 記憶體需求：114 GB → 20 GB
- 訓練速度：1x → 3-5x
- 儲存成本：14 GB/任務 → 10 MB/任務
- 效果損失：通常 < 5%

**LLM 的 fine-tuning 發生在低秩（low-rank）子空間中**

### 基本想法

原始模型：W ∈ R^(d×k)  (例如 4096×4096)

**Full Fine-tuning**： W_new = W + ΔW

- 其中 ΔW 的每個元素都可能改變

**PEFT**：W_new = W + low_rank_adaptation

- 其中 adaptation 只有很少的自由度

### 為什麼low-rank假設有效？
假設我們有一個已經訓練好的大模型，它的權重叫做 W_original。現在我們拿這個模型去做全參數 fine-tune，得到一組新的權重 W_finetuned。如果我們把兩者相減，得到的差值：
$$ΔW = W_{finetuned} − W_{original}$$

過去的實驗發現，這個 ΔW 雖然看起來是一個超大的矩陣（例如 4096×4096），理論上rank可以高達 4096，但實際上並不是這樣。你如果對 ΔW 做 SVD，會發現前面只有非常少數幾個奇異值就已經解釋了幾乎全部的變化量。像是前 16 個奇異值，就能吃掉 95% 以上的能量。

用白話講就是：模型在 fine-tune 的時候，真的有在學的新東西，其實集中在一個很低維的方向空間裡。它並沒有用滿整個 4096 維的自由度，而是只在其中一小撮「重要方向」上做調整。

這個觀察就是 LoRA 的核心直覺。既然真正有用的更新只活在一個低維子空間，那我們幹嘛還要傻傻地更新整個 4096×4096 的權重矩陣？參數量爆炸、VRAM又貴，完全沒必要。

**LoRA 的做法就是**：原本的權重 W_original 我們凍結不動，然後額外加上一個「低秩更新項」。這個更新項通常被拆成兩個小矩陣相乘，一個是把高維空間壓到低維，另一個再把低維投回原本的維度。這樣一來，我們實際學的參數量就只跟那個低維的 rank 有關，比如 8、16、32，而不是 4096²。

### PEFT 的統一框架
**所有 PEFT 方法都遵循相同的模式：**

In [ ]:
class PEFTMethod:
    def __init__(self, pretrained_model):
        self.backbone = pretrained_model
        self.backbone.requires_grad_(False)  # 凍結
        
        self.trainable_params = self.create_efficient_params()
    
    def forward(self, x):
        # 原始模型的輸出
        base_output = self.backbone(x)
        
        # PEFT 的調整
        adaptation = self.apply_efficient_params(x)
        
        # 組合
        return base_output + adaptation
    
    def create_efficient_params(self):
        """各種 PEFT 方法的差異在這裡"""
        raise NotImplementedError

## 主流 PEFT 方法分類
### 方法概覽

**Additive Methods (加法式)**
- Adapter

    └─ 在層之間插入小型可訓練模組
- Prefix Tuning

    └─ 在輸入層添加可訓練的前綴向量
- Prompt Tuning

    └─ 只在最前面加可訓練的 soft prompt

**Selective Methods (選擇式)**
- BitFit
    └─ 只訓練 bias 參數

- Partial Fine-tuning
    └─ 只訓練特定層（如最後幾層）

**Reparameterization Methods (重參數化)**
- LoRA (Low-Rank Adaptation)
    └─ 用低秩矩陣分解來近似 ΔW

### 方法比較表

| 方法 | 可訓練參數量 | 推理延遲 | 記憶體效率 | 效果 | 實作複雜度 |
|------|------------|---------|-----------|------|-----------|
| **LoRA** | 0.1-1% | 無 | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | 低 |
| **Adapter** | 0.5-2% | 有 | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ | 中 |
| **Prefix Tuning** | 0.01-0.1% | 有 | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | 中 |
| **Prompt Tuning** | <0.01% | 有 | ⭐⭐⭐⭐⭐ | ⭐⭐ | 低 |

## LoRA：為什麼成為主流？
### 核心想法

**LoRA (Low-Rank Adaptation) 的想法極其簡單：**

**原始權重矩陣**： W ∈ R^(d×k)

**Full Fine-tuning**：W' = W + ΔW  (ΔW 是 d×k 的滿秩矩陣)

**LoRA**：W' = W + BA

其中：
- B ∈ R^(d×r)
- A ∈ R^(r×k)
- r << min(d, k)  (rank，通常 r=8,16,32)

**參數量比較：**
d=4096, k=4096, r=16

**Full**: d × k = 16,777,216 參數

**LoRA**: d × r + r × k = r(d + k) = 131,072 參數 (0.78%)

### 為什麼 LoRA 這麼受歡迎？

**優勢 1：零推理延遲**

LoRA 在訓練時會多學一組小矩陣，但在部署前可以直接把這些權重合併回原本的模型，只需要做一次合併就好。合併之後，推理時的計算流程和原本模型完全一樣，不會多出任何額外運算，因此幾乎沒有推理延遲。這一點跟 Adapter 很不一樣，Adapter 在推理時還得多走一次前向傳播，自然就會拖慢速度。

**優勢 2：模組化切換**

In [ ]:
# 多任務部署
base_model = load_pretrained()

# 載入不同任務的 LoRA 權重
lora_A_task1, lora_B_task1 = load_lora("task1")
lora_A_task2, lora_B_task2 = load_lora("task2")

# 快速切換
def switch_task(task_id):
    if task_id == 1:
        apply_lora(lora_A_task1, lora_B_task1)
    elif task_id == 2:
        apply_lora(lora_A_task2, lora_B_task2)


# 儲存空間
"""
Base model: 14 GB
LoRA per task: 10-50 MB
10 個任務: 14 GB + 10 × 50 MB = 14.5 GB
vs Full FT: 10 × 14 GB = 140 GB
"""

**優勢 3：訓練穩定**

LoRA 的初始化設計讓訓練一開始不會影響原本的預訓練權重，模型等於是從原狀態起跑，再慢慢學會怎麼調整。這樣可以避免訓練初期的不穩定，讓模型用更平滑、漸進的方式進行微調。

# To Be Continued